In [1]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

In [2]:
# Setup paths
#PROJECT_ROOT = Path(__file__).parent.parent
PROJECT_ROOT = Path.cwd().parent
DATA_PATH = PROJECT_ROOT / "data" / "raw" / "Test_Data.xlsx"

print(PROJECT_ROOT)
print(DATA_PATH)


c:\Users\PC\Documents\DE_Projects\bidco-retail-analysis
c:\Users\PC\Documents\DE_Projects\bidco-retail-analysis\data\raw\Test_Data.xlsx


In [3]:

# Load the data
df = pl.read_excel(DATA_PATH)

In [4]:
print(len(df))  
df.columns

##30691 ROWS
#13 Columns

30691


['Store Name',
 'Item_Code',
 'Item Barcode',
 'Description',
 'Category',
 'Department',
 'Sub-Department',
 'Section',
 'Quantity',
 'Total Sales',
 'RRP',
 'Supplier',
 'Date Of Sale']

In [5]:
#Basic Infomation to understand the data
print(df.schema)

Schema({'Store Name': String, 'Item_Code': Int64, 'Item Barcode': String, 'Description': String, 'Category': String, 'Department': String, 'Sub-Department': String, 'Section': String, 'Quantity': Float64, 'Total Sales': Float64, 'RRP': Float64, 'Supplier': String, 'Date Of Sale': Date})


In [6]:
print("First few rows:")
print(df.head())
print()

First few rows:
shape: (5, 13)
┌────────┬───────────┬────────────┬────────────┬───┬────────────┬────────┬────────────┬────────────┐
│ Store  ┆ Item_Code ┆ Item       ┆ Descriptio ┆ … ┆ Total      ┆ RRP    ┆ Supplier   ┆ Date Of    │
│ Name   ┆ ---       ┆ Barcode    ┆ n          ┆   ┆ Sales      ┆ ---    ┆ ---        ┆ Sale       │
│ ---    ┆ i64       ┆ ---        ┆ ---        ┆   ┆ ---        ┆ f64    ┆ str        ┆ ---        │
│ str    ┆           ┆ str        ┆ str        ┆   ┆ f64        ┆        ┆            ┆ date       │
╞════════╪═══════════╪════════════╪════════════╪═══╪════════════╪════════╪════════════╪════════════╡
│ SHABAB ┆ 280236    ┆ 6374692674 ┆ HC-TOPEX   ┆ … ┆ 103.448196 ┆ 91.41  ┆ SUPERSLEEK ┆ 2025-09-23 │
│        ┆           ┆ 377        ┆ LEMON      ┆   ┆            ┆        ┆ LIMITED    ┆            │
│        ┆           ┆            ┆ 250ML      ┆   ┆            ┆        ┆            ┆            │
│ SHABAB ┆ 280255    ┆ 2541751409 ┆ HC-TKL JIK ┆ … ┆ 452.585

In [7]:
print("Summary statistics:")
print(df.describe())
print() 

Summary statistics:
shape: (9, 14)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ Store     ┆ Item_Code ┆ Item      ┆ … ┆ Total     ┆ RRP       ┆ Supplier  ┆ Date Of  │
│ ---       ┆ Name      ┆ ---       ┆ Barcode   ┆   ┆ Sales     ┆ ---       ┆ ---       ┆ Sale     │
│ str       ┆ ---       ┆ f64       ┆ ---       ┆   ┆ ---       ┆ f64       ┆ str       ┆ ---      │
│           ┆ str       ┆           ┆ str       ┆   ┆ f64       ┆           ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 30691     ┆ 30691.0   ┆ 30691     ┆ … ┆ 30691.0   ┆ 30659.0   ┆ 30676     ┆ 30691    │
│ null_coun ┆ 0         ┆ 0.0       ┆ 0         ┆ … ┆ 0.0       ┆ 32.0      ┆ 15        ┆ 0        │
│ t         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ mean      ┆ null      ┆ 493417.91 ┆ null      ┆ … ┆ 45

In [8]:
##checking for Null Data
null_counts = df.null_count()
print("Missing values per column:")
for col in df.columns:
    null_count = null_counts[col][0]
    null_pct = (null_count / len(df)) * 100
    if null_count > 0:
        print(f"  {col}: {null_count:,} ({null_pct:.2f}%)")
print()


Missing values per column:
  RRP: 32 (0.10%)
  Supplier: 15 (0.05%)



In [9]:
# Check for duplicates
print("Checking for duplicates...")
duplicate_count = len(df) - df.n_unique(subset=df.columns)
if duplicate_count > 0:
    print(f" Found {duplicate_count:,} duplicate rows")
else:
    print(" No duplicate rows found")
print()

Checking for duplicates...
 No duplicate rows found



In [10]:
# Date range
if "Date Of Sale" in df.columns:
    print("Date range:")
    date_stats = df.select([
        pl.col("Date Of Sale").min().alias("Start Date"),
        pl.col("Date Of Sale").max().alias("End Date"),
        pl.col("Date Of Sale").n_unique().alias("Unique Days")
    ])
    print(date_stats)
    print()

# Unique counts for key dimensions
print("DIMENSION CARDINALITY")
dimension_cols = ["Store Name", "Supplier", "Category", "Department", "Sub-Department", "Section"]
for col in dimension_cols:
    if col in df.columns:
        unique_count = df[col].n_unique()
        print(f"  {col}: {unique_count:,} unique values")
print()

Date range:
shape: (1, 3)
┌────────────┬────────────┬─────────────┐
│ Start Date ┆ End Date   ┆ Unique Days │
│ ---        ┆ ---        ┆ ---         │
│ date       ┆ date       ┆ u32         │
╞════════════╪════════════╪═════════════╡
│ 2025-09-22 ┆ 2025-09-28 ┆ 7           │
└────────────┴────────────┴─────────────┘

DIMENSION CARDINALITY
  Store Name: 35 unique values
  Supplier: 190 unique values
  Category: 3 unique values
  Department: 11 unique values
  Sub-Department: 52 unique values
  Section: 209 unique values



In [11]:
##focusing on Bidco


if "Supplier" in df.columns:
    # Find Bidco
    suppliers = df.select("Supplier").unique().sort("Supplier")
    print("All suppliers:")
    print(suppliers)
    print()
    
    # Check if Bidco exists
    bidco_df = df.filter(pl.col("Supplier").str.contains("(?i)bidco"))
    print(f"Bidco rows: {len(bidco_df):,} ({len(bidco_df)/len(df)*100:.2f}%)")
    
    if len(bidco_df) > 0:
        print()
        print("Bidco product categories:")
        bidco_categories = bidco_df.group_by("Category").agg([
            pl.count().alias("Row Count"),
            pl.col("Total Sales").sum().alias("Total Sales")
        ]).sort("Total Sales", descending=True)
        print(bidco_categories)
print()

All suppliers:
shape: (190, 1)
┌───────────────────────────┐
│ Supplier                  │
│ ---                       │
│ str                       │
╞═══════════════════════════╡
│ null                      │
│ ALISON PRODUCTS KENYA LTD │
│ ALMASI BOTTLERS LIMITED   │
│ AQUAMIST LIMITED          │
│ ARYUV AGENCIES LIMITED    │
│ …                         │
│ WISH KENYA LIMITED        │
│ WOODLANDS COMPANY LIMITED │
│ WORLD BRANDS (K) LIMITED  │
│ YASTECH BRANDS LIMITED    │
│ ZENKO KENYA LIMITED       │
└───────────────────────────┘

Bidco rows: 1,000 (3.26%)

Bidco product categories:
shape: (3, 3)
┌───────────────┬───────────┬───────────────┐
│ Category      ┆ Row Count ┆ Total Sales   │
│ ---           ┆ ---       ┆ ---           │
│ str           ┆ u32       ┆ f64           │
╞═══════════════╪═══════════╪═══════════════╡
│ FOODS         ┆ 844       ┆ 816416.936594 │
│ HOMECARE      ┆ 155       ┆ 282724.031937 │
│ PERSONAL CARE ┆ 1         ┆ 1051.723999   │
└───────────────┴──────

C:\Users\PC\AppData\Local\Temp\ipykernel_20076\3891424729.py:19: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("Row Count"),


In [12]:
# Sales metrics overview

if "Total Sales" in df.columns and "Quantity" in df.columns:
    sales_summary = df.select([
        pl.col("Total Sales").sum().alias("Total Sales Value"),
        pl.col("Quantity").sum().alias("Total Units"),
        #pl.col("Total Sales") / pl.col("Quantity")).mean().alias("Avg Unit Price")
    ])
    print(sales_summary)
    print()
    
    # Check for negative values
    negative_qty = df.filter(pl.col("Quantity") < 0)
    negative_sales = df.filter(pl.col("Total Sales") < 0)
    
    print(f"Negative quantities: {len(negative_qty):,} rows")
    print(f"Negative sales: {len(negative_sales):,} rows")
    print()
    
    # Zero values
    zero_qty = df.filter(pl.col("Quantity") == 0)
    zero_sales = df.filter(pl.col("Total Sales") == 0)
    
    print(f"Zero quantities: {len(zero_qty):,} rows")
    print(f"Zero sales: {len(zero_sales):,} rows")
print()

shape: (1, 2)
┌───────────────────┬─────────────┐
│ Total Sales Value ┆ Total Units │
│ ---               ┆ ---         │
│ f64               ┆ f64         │
╞═══════════════════╪═════════════╡
│ 1.4020e7          ┆ 103099.975  │
└───────────────────┴─────────────┘

Negative quantities: 5 rows
Negative sales: 5 rows

Zero quantities: 12 rows
Zero sales: 12 rows



In [13]:
# PRICING OVERVIEW"


if all(col in df.columns for col in ["Total Sales", "Quantity", "RRP"]):
    # Calculate realized unit price
    df_price = df.with_columns([
        (pl.col("Total Sales") / pl.col("Quantity")).alias("Realized_Unit_Price")
    ]).filter(
        (pl.col("Quantity") > 0) & (pl.col("Total Sales") > 0)
    )
    
    price_stats = df_price.select([
        pl.col("Realized_Unit_Price").min().alias("Min Price"),
        pl.col("Realized_Unit_Price").quantile(0.25).alias("P25 Price"),
        pl.col("Realized_Unit_Price").median().alias("Median Price"),
        pl.col("Realized_Unit_Price").quantile(0.75).alias("P75 Price"),
        pl.col("Realized_Unit_Price").max().alias("Max Price"),
        pl.col("RRP").median().alias("Median RRP"),
    ])
    
    print("Price distribution:")
    print(price_stats)
    print()
    
    # Check for extreme prices
    price_threshold = df_price["Realized_Unit_Price"].quantile(0.99)
    extreme_prices = df_price.filter(pl.col("Realized_Unit_Price") > price_threshold)
    print(f"Prices above 99th percentile (>{price_threshold:.2f}): {len(extreme_prices):,} rows")
print()

Price distribution:
shape: (1, 6)
┌───────────┬───────────┬──────────────┬────────────┬────────────┬────────────┐
│ Min Price ┆ P25 Price ┆ Median Price ┆ P75 Price  ┆ Max Price  ┆ Median RRP │
│ ---       ┆ ---       ┆ ---          ┆ ---        ┆ ---        ┆ ---        │
│ f64       ┆ f64       ┆ f64          ┆ f64        ┆ f64        ┆ f64        │
╞═══════════╪═══════════╪══════════════╪════════════╪════════════╪════════════╡
│ 3.4483    ┆ 54.310299 ┆ 116.379303   ┆ 214.655106 ┆ 8735.34375 ┆ 110.735    │
└───────────┴───────────┴──────────────┴────────────┴────────────┴────────────┘

Prices above 99th percentile (>1325.86): 296 rows



In [14]:
# Store-level analysis


if "Store Name" in df.columns:
    store_summary = df.group_by("Store Name").agg([
        pl.len().alias("Transaction Count"),
        pl.col("Total Sales").sum().alias("Total Sales"),
        pl.col("Quantity").sum().alias("Total Units"),
        pl.col("Supplier").n_unique().alias("Unique Suppliers"),
        pl.col("Category").n_unique().alias("Unique Categories")
    ]).sort("Total Sales", descending=True)
    
    print(f"Total stores: {len(store_summary)}")
    print()
    print("Top 10 stores by sales:")
    print(store_summary.head(10))
print()

Total stores: 35

Top 10 stores by sales:
shape: (10, 6)
┌────────────┬─────────────┬───────────────┬─────────────┬────────────────────┬────────────────────┐
│ Store Name ┆ Transaction ┆ Total Sales   ┆ Total Units ┆ Unique Suppliers   ┆ Unique Categories  │
│ ---        ┆ Count       ┆ ---           ┆ ---         ┆ ---                ┆ ---                │
│ str        ┆ ---         ┆ f64           ┆ f64         ┆ u32                ┆ u32                │
│            ┆ u32         ┆               ┆             ┆                    ┆                    │
╞════════════╪═════════════╪═══════════════╪═════════════╪════════════════════╪════════════════════╡
│ KIAMBU RD  ┆ 1134        ┆ 757639.772925 ┆ 4978.665    ┆ 116                ┆ 3                  │
│ MFANGANO   ┆ 1200        ┆ 757163.629546 ┆ 7316.41     ┆ 124                ┆ 3                  │
│ KILIMANI   ┆ 1330        ┆ 739060.407947 ┆ 4813.595    ┆ 132                ┆ 3                  │
│ LAVINGTON  ┆ 1146        ┆ 69049

Data Exploration Summary


Size: 30,691 rows × 13 columns
Time Period: 7 days (Sept 22-28, 2025)
Stores: 35 unique retail outlets
Suppliers: 190 unique suppliers
Total Sales Value: ~14M in sales value

##Key Findings

Only 32 missing RRP values (0.10%)
Only 15 missing Supplier values (0.05%)
No duplicate rows
All 13 expected columns present

Issues to handle:

5 rows with negative quantities/sales (returns?)
12 rows with zero quantities/sales
296 rows with extreme prices (>99th percentile at 1,325.86)
Some zero barcodes (need to investigate)

1. Bidco Presence

1,000 rows (3.26% of dataset)
3 Categories: Foods (844 rows), Homecare (155 rows), Personal Care (1 row)
Total Bidco Sales: ~1.1M (7.8% of total market sales)
Bidco is present but not dominant 

2. Market Structure
Categories: 3 main categories (Foods, Homecare, Personal Care)
Departments: 11 departments
Sub-Departments: 52 
Sections: 209 (granular product segments)
Insight: Sub-Department + Section gives us precise competitive sets for pricing analysis

3. Store Distribution

All stores carry multiple categories (3 each)
Store sales range: ~200K to ~760K per week
Top stores: Kiambu Rd, Mfangano, Kilimani
Even distribution suggests consistent data collection

4. Pricing Patterns (Critical for Promo Detection)

5. Realized Unit Price (Total Sales / Quantity):

Median: 116.38
P25: 54.31, P75: 214.66
Range: 3.45 to 8,735.34 (huge variance)


6. RRP (Recommended Retail Price):

Median: 110.74

